# Chapter 2: Semirings

Different semirings give different meanings to matrix operations on the same graph.

In [ ]:
import graphblas as gb
from graphblas import Matrix, Vector, semiring
import networkx as nx
import matplotlib.pyplot as plt

## Weighted Graph

In [ ]:
# Create a weighted graph
rows = [0, 0, 1, 2, 3]
cols = [1, 2, 2, 3, 1]
weights = [4, 2, 3, 1, 5]

A = Matrix.from_coo(rows, cols, weights, nrows=4, ncols=4, dtype=float)
print("Weighted adjacency matrix:")
print(A)

In [ ]:
# Visualize
G = nx.DiGraph()
for r, c, w in zip(rows, cols, weights):
    G.add_edge(r, c, weight=w)

pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=16, arrows=True)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.show()

## PLUS_TIMES Semiring

Standard arithmetic: multiply weights, sum paths. Counts weighted paths.

In [ ]:
v = Vector.from_coo([0], [1.0], size=4)
result = v.mxv(A.T, semiring.plus_times).new()
print("PLUS_TIMES from node 0:")
print(result)
print("\nInterpretation: edge weight to each neighbor")

## MIN_PLUS Semiring

Tropical semiring: add weights, take minimum. Finds shortest paths.

In [ ]:
v = Vector.from_coo([0], [0.0], size=4)  # Start with distance 0
result = v.mxv(A.T, semiring.min_plus).new()
print("MIN_PLUS from node 0:")
print(result)
print("\nInterpretation: shortest distance to each neighbor (1 hop)")

In [ ]:
# Two hops with MIN_PLUS
result2 = result.mxv(A.T, semiring.min_plus).new()
print("MIN_PLUS after 2 hops:")
print(result2)

## ANY_PAIR Semiring

Ignores weights entirely. Just checks connectivity (any edge = 1).

In [ ]:
v = Vector.from_coo([0], [True], size=4, dtype=bool)
A_bool = Matrix.from_coo(rows, cols, [True]*len(rows), nrows=4, ncols=4, dtype=bool)

result = v.mxv(A_bool.T, semiring.any_pair).new()
print("ANY_PAIR from node 0:")
print(result)
print("\nInterpretation: which nodes are reachable (ignoring weights)")

## Summary

| Semiring | Multiply | Add | Use Case |
|----------|----------|-----|----------|
| PLUS_TIMES | × | + | Path counting, standard LA |
| MIN_PLUS | + | min | Shortest paths |
| ANY_PAIR | 1 | any | Reachability, BFS |